In [ ]:
# Installing necessary libraries
!pip install -q wordcloud emoji seaborn matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 6.8 MB/s eta 0:00:00


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter
from wordcloud import WordCloud
import emoji
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Set visualization style
sns.set_theme(style="whitegrid")

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving IMDB_Dataset.csv to IMDB_Dataset.csv


In [ ]:
# Loading the dataset into pandas DataFrame
df = pd.read_csv('IMDB_Dataset.csv')

## ***LLM***

# **DISTIL-BERT**

### **Step1: Load the necessary Libraries**

In [ ]:
import torch
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from torch.cuda.amp import autocast, GradScaler

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### **Step2: Pre-processing the data**

In [ ]:
df["sentiment_label"] = df["sentiment"].map({"negative": 0, "neutral": 1, "positive": 2})

In [ ]:
df.shape

(50000, 3)

In [ ]:
df.head(5)

,review,sentiment,sentiment_label
0,One of the other reviewers has mentioned that ...,positive,2
1,A wonderful little production. <br /><br />The...,positive,2
2,I thought this was a wonderful way to spend ti...,positive,2
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,2


In [ ]:
df_sample = df.sample(5000)

### **Step3: Split Data into Training and Testing Sets**

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
df_sample["review"].tolist(), df_sample["sentiment_label"].tolist(), test_size=0.2, random_state=42)

### **Step4: Load DistilBERT Tokenizer**

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

### **Step5: Define a PyTorch Dataset Class**

In [ ]:
class IMDBDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long),
        }

### **Step6: Create DataLoaders**

In [ ]:
train_dataset = IMDBDataset(train_texts, train_labels, tokenizer)
test_dataset = IMDBDataset(test_texts, test_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

### **Step7: Load Pretrained DistilBERT Model**

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)
model.to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


### **Step8: Define Optimizer and Loss Function**

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()
scaler = GradScaler()  # Mixed precision training

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-24-1b79480157db>:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # Mixed precision training
/usr/local/lib/python3.11/dist-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


### **Step9: Train the Model with Gradient Accumulation**

In [ ]:
gradient_accumulation_steps = 4
num_epochs = 2

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for i, batch in enumerate(train_loader):
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with autocast():
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = criterion(outputs.logits, labels)

        loss = loss / gradient_accumulation_steps
        scaler.scale(loss).backward()

        if (i + 1) % gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}: Loss = {total_loss / len(train_loader):.4f}")

<ipython-input-26-8fdb69cdab05>:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1: Loss = 0.1184
Epoch 2: Loss = 0.0797


In [ ]:
# Save model
torch.save(model.state_dict(), "distilbert_imdb_model.pth")

# **BERT-base Uncased**

In [ ]:
pip install transformers datasets torch scikit-learn pandas numpy

### **Step 1: Install Required Libraries**

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

### **Step 2: Load and Preprocess the Dataset**

In [ ]:
# Convert sentiment labels to numeric values
label_mapping = {"negative": 0, "neutral": 1, "positive": 2}
df["sentiment_label"] = df["sentiment"].map(label_mapping)

In [ ]:
print("Dataset Overview:")
print(df.head())

Dataset Overview:
                                              review sentiment  \
0  One of the other reviewers has mentioned that ...  positive   
1  A wonderful little production. <br /><br />The...  positive   
2  I thought this was a wonderful way to spend ti...  positive   
3  Basically there's a family where a little boy ...  negative   
4  Petter Mattei's "Love in the Time of Money" is...  positive   

   sentiment_label  
0                2  
1                2  
2                2  
3                0  
4                2  


In [ ]:
# Split into training and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["review"].tolist(), df["sentiment_label"].tolist(), test_size=0.2, random_state=42
)

In [ ]:
# Load BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### **Step 3: Tokenizing the Text Data**

In [ ]:
# Tokenize the text data
def tokenize_function(texts):
    return tokenizer(
        texts, padding="max_length", truncation=True, max_length=256, return_tensors="pt"
    )

train_encodings = tokenize_function(train_texts)
test_encodings = tokenize_function(test_texts)

### **Step 4: Convert Data to PyTorch Format**

In [ ]:
# Convert labels to tensor format
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

### **Step 5: Create a PyTorch Dataset and DataLoader**

In [ ]:
from torch.utils.data import Dataset, DataLoader

# Define a dataset class
class IMDBDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            key: tensor[idx] for key, tensor in self.encodings.items()
        } | {"labels": self.labels[idx]}

# Create dataset objects
train_dataset = IMDBDataset(train_encodings, train_labels)
test_dataset = IMDBDataset(test_encodings, test_labels)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4)


### **Step 6: Load BERT for Multi-Class Classification**

In [ ]:
from transformers import AutoModelForSequenceClassification

# Load BERT model with a classification head
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### **Step 7: Define Optimizer and Loss Function**

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)


### **Step 8: Train the Model**

In [ ]:
from torch.cuda.amp import autocast, GradScaler

gradient_accumulation_steps = 4
scaler = GradScaler()

for epoch in range(3):
    model.train()
    total_loss = 0

    for i, batch in enumerate(train_loader):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        with autocast():  # Enable mixed precision
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = criterion(outputs.logits, labels)

        loss = loss / gradient_accumulation_steps  # Scale loss
        scaler.scale(loss).backward()  # Use mixed precision

        if (i + 1) % gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}: Loss = {total_loss / len(train_loader):.4f}")


<ipython-input-24-f2566068a94f>:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/usr/local/lib/python3.11/dist-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
<ipython-input-24-f2566068a94f>:16: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Enable mixed precision
/usr/local/lib/python3.11/dist-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


### **Step 9: Evaluate Model**

In [ ]:
from sklearn.metrics import accuracy_score

model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=-1).cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
print(f"Test Accuracy: {accuracy:.4f}")


### **Step 10: Save the Model**

In [ ]:
print("\nDataset Info:")
print(df.info())


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB
None


In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing Values:\n", missing_values)

Missing Values:
 review       0
sentiment    0
dtype: int64


In [ ]:
df.describe().style.background_gradient(cmap='tab20c')

,review,sentiment
count,50000,50000
unique,49582,2
top,"Loved today's show!!! It was a variety and not solely cooking (which would have been great too). Very stimulating and captivating, always keeping the viewer peeking around the corner to see what was coming up next. She is as down to earth and as personable as you get, like one of us which made the show all the more enjoyable. Special guests, who are friends as well made for a nice surprise too. Loved the 'first' theme and that the audience was invited to play along too. I must admit I was shocked to see her come in under her time limits on a few things, but she did it and by golly I'll be writing those recipes down. Saving time in the kitchen means more time with family. Those who haven't tuned in yet, find out what channel and the time, I assure you that you won't be disappointed.",positive
freq,5,25000


In [ ]:
df['sentiment_num'] = [1 if sentiment == 'positive' else 0 for sentiment in df['sentiment']]
df.head()

,review,sentiment,sentiment_num
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1


### ***Bag of Words***

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

from tqdm import tqdm
corpus = []
for i in tqdm(range(0,len(df))):
    sentence = re.sub('[^a-zA-Z]',' ',df['review'][i]) ## each review is each sentence
    sentence = sentence.lower() ## Lower casing the words in each sentence
    sentence = sentence.split() ## splitting sentences to words and storing it as a list of words
    sentence = [ps.stem(word) for word in sentence if not word in stopwords.words('english')]  ## Removing stop words and applying stemming
    sentence = ' '.join(sentence)  ## Joining words again to form the sentences
    corpus.append(sentence) ### storing each sentences to corpus

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
100%|██████████| 50000/50000 [27:42<00:00, 30.07it/s]


In [ ]:
ps.stem('running')

'run'

In [ ]:
## Creating bag of words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
X1 = cv.fit_transform(corpus).toarray()

In [ ]:
y1 = pd.get_dummies(df['sentiment'])
y1 = y1.iloc[:,1].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1,y1,test_size = 0.20, random_state=0)

In [ ]:
X_train1.shape

(40000, 2500)

In [ ]:
y_train1.shape

(40000,)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model1 = MultinomialNB().fit(X_train1,y_train1)

In [ ]:
y_pred1 = model1.predict(X_test1)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(y_test1,y_pred1))
print(classification_report(y_pred1,y_test1))

0.8369
              precision    recall  f1-score   support

       False       0.84      0.84      0.84      5052
        True       0.83      0.84      0.84      4948

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000



In [ ]:
new_review = 'I love this movie so much. It\'s really great'
new_review = re.sub('[^a-zA-Z]', ' ', new_review)
new_review = new_review.lower()
new_review = new_review.split()
ps = PorterStemmer()
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')
new_review = [ps.stem(word) for word in new_review if not word in set(all_stopwords)]
new_review = ' '.join(new_review)
new_corpus = [new_review]
new_X_test = cv.transform(new_corpus).toarray()
new_y_pred = model1.predict(new_X_test)
print(new_y_pred)

[ True]


## ***TF-IDF***

In [ ]:
## Creating tf-idf model
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features=2500)
X2 = tv.fit_transform(corpus).toarray()

In [ ]:
y2=pd.get_dummies(df['sentiment'])
y2=y2.iloc[:,1].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2,y2, test_size=0.20, random_state=0)

In [ ]:
X_train2.shape

(40000, 2500)

In [ ]:
y_train2.shape

(40000,)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model2 = MultinomialNB().fit(X_train2, y_train2)

In [ ]:
y_pred2 = model2.predict(X_test2)

In [ ]:
print(accuracy_score(y_test2,y_pred2))
print(classification_report(y_pred2,y_test2))

0.8448
              precision    recall  f1-score   support

       False       0.83      0.85      0.84      4923
        True       0.85      0.84      0.85      5077

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.85      0.84      0.84     10000



In [ ]:
new_review = 'I Hate this movie so much. It\'s ok.'
new_review = re.sub('[^a-zA-Z]', ' ', new_review)
new_review = new_review.lower()
new_review = new_review.split()
ps = PorterStemmer()
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')
new_review = [ps.stem(word) for word in new_review if not word in set(all_stopwords)]
new_review = ' '.join(new_review)
new_corpus = [new_review]
new_X_test = cv.transform(new_corpus).toarray()
new_y_pred = model2.predict(new_X_test)
print(new_y_pred)

[False]


# ***Word2Vec using RF***

In [ ]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

# Download stopwords and lemmatizer if not already downloaded
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Preprocessing function
def preprocess_text(text):
    # 1. Convert to lowercase
    text = text.lower()

    # 2. Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # 3. Remove punctuation and numbers
    text = re.sub(r'[^\w\s]', '', text)  # Removes punctuation
    text = re.sub(r'\d+', '', text)     # Removes numbers

    # 4. Tokenize (split text into words)
    tokens = text.split()

    # 5. Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]

    # 6. Lemmatize words
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Join tokens back into a single string
    return ' '.join(tokens)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Apply preprocessing to the review column
df['review_clean'] = df['review'].apply(preprocess_text)

In [ ]:
df.head(5)

,review,sentiment,sentiment_num,review_clean
0,One of the other reviewers has mentioned that ...,positive,1,one reviewer mentioned watching oz episode you...
1,A wonderful little production. <br /><br />The...,positive,1,wonderful little production filming technique ...
2,I thought this was a wonderful way to spend ti...,positive,1,thought wonderful way spend time hot summer we...
3,Basically there's a family where a little boy ...,negative,0,basically there family little boy jake think t...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1,petter matteis love time money visually stunni...


In [ ]:
import os
print(os.getcwd())

/content


In [ ]:
import os
os.listdir('/content')

['.config', 'IMDB_Dataset.csv', 'sample_data']

In [ ]:
import nltk
nltk.download('punkt')  # Correctly download 'punkt' tokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import nltk
print(nltk.data.find('tokenizers/punkt'))

/root/nltk_data/tokenizers/punkt


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import numpy as np
import joblib  # Import joblib for saving the model
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import re

# Assuming df contains the preprocessed text ('review_clean') and sentiment ('sentiment')
X = df['review_clean']
y = df['sentiment']

In [ ]:
def tokenize(text):
    # Remove anything that's not a letter and lowercase the text
    return word_tokenize(re.sub('[^a-zA-Z]', ' ', text.lower()))  # Make sure punkt is loaded first

In [ ]:
import nltk
nltk.data.path.append('/usr/local/share/nltk_data')

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

def tokenize_spacy(text):
    doc = nlp(text)
    return [token.text for token in doc if token.is_alpha]  # Only keep alphabetic tokens

# Apply spacy tokenization to the 'review_clean' column
X_tokenized = X.apply(tokenize_spacy)


In [ ]:
# Train a Word2Vec model on the tokenized reviews (if you don't have a pre-trained one)
word2vec_model = Word2Vec(sentences=X_tokenized, vector_size=100, window=5, min_count=1, workers=4)

# Save the Word2Vec model (optional)
word2vec_model.save('word2vec_model')  # Save the model to disk

# Function to average Word2Vec embeddings for each review
def get_average_word2vec(review, model, vector_size=100):
    # Get Word2Vec vectors for words in review
    words = review
    word_vecs = [model.wv[word] for word in words if word in model.wv]
    if len(word_vecs) == 0:
        return np.zeros(vector_size)  # Return zero vector if no words match in Word2Vec
    return np.mean(word_vecs, axis=0)

In [ ]:
# Convert tokenized reviews to average Word2Vec embeddings
X_word2vec = np.array([get_average_word2vec(review, word2vec_model) for review in X_tokenized])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_word2vec, y, test_size=0.2, random_state=42)

# Train RandomForest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.84      0.82      0.83      4961
    positive       0.82      0.85      0.84      5039

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



In [ ]:
# Save the RandomForest model
joblib.dump(rf_model, 'random_forest_model_word2vec.pkl')  # Save the trained RandomForest model
joblib.dump(word2vec_model, 'word2vec_model.pkl')  # Save the Word2Vec model (if needed)

['word2vec_model.pkl']

In [ ]:
# Sample new review to test
new_review = "This movie was absolutely amazing!"

# Tokenize and process the new review (using Spacy)
new_review_tokenized = tokenize_spacy(new_review)

# Get average Word2Vec embeddings for the new review
new_review_vector = get_average_word2vec(new_review_tokenized, word2vec_model)

# Reshape the vector to match the expected input format for RandomForest
new_review_vector = new_review_vector.reshape(1, -1)

# Predict sentiment
new_y_pred = rf_model.predict(new_review_vector)
print("Predicted sentiment:", new_y_pred)

Predicted sentiment: ['positive']


In [ ]:
# Sample new review to test
new_review = "The movie was very pathetic!"

# Tokenize and process the new review (using Spacy)
new_review_tokenized = tokenize_spacy(new_review)

# Get average Word2Vec embeddings for the new review
new_review_vector = get_average_word2vec(new_review_tokenized, word2vec_model)

# Reshape the vector to match the expected input format for RandomForest
new_review_vector = new_review_vector.reshape(1, -1)

# Predict sentiment
new_y_pred = rf_model.predict(new_review_vector)
print("Predicted sentiment:", new_y_pred)

Predicted sentiment: ['negative']


In [ ]:
# Sample new review to test
new_review = "Movie could have been better!"

# Tokenize and process the new review (using Spacy)
new_review_tokenized = tokenize_spacy(new_review)

# Get average Word2Vec embeddings for the new review
new_review_vector = get_average_word2vec(new_review_tokenized, word2vec_model)

# Reshape the vector to match the expected input format for RandomForest
new_review_vector = new_review_vector.reshape(1, -1)

# Predict sentiment
new_y_pred = rf_model.predict(new_review_vector)
print("Predicted sentiment:", new_y_pred)

Predicted sentiment: ['negative']


## ***Word2Vec using GaussianNB***

In [ ]:
# Required Libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from gensim.models import Word2Vec
import spacy

In [ ]:
# Load SpaCy for Tokenization
nlp = spacy.load('en_core_web_sm')

# Tokenize Function (using SpaCy)
def tokenize(text):
    doc = nlp(text.lower())
    return [token.text for token in doc if token.is_alpha]  # Keep only alphabetic words

In [ ]:
# Tokenize the reviews
df['tokenized'] = df['review'].apply(tokenize)

# Train Word2Vec Model
word2vec_model = Word2Vec(sentences=df['tokenized'], vector_size=100, window=5, min_count=1, workers=4)


In [ ]:
# Save Word2Vec Model (Optional)
word2vec_model.save('word2vec_model')


In [ ]:
# Function to Average Word2Vec Embeddings
def get_average_word2vec(tokens, model, vector_size=100):
    word_vecs = [model.wv[word] for word in tokens if word in model.wv]
    if len(word_vecs) == 0:
        return np.zeros(vector_size)  # If no words in vocab, return a zero vector
    return np.mean(word_vecs, axis=0)


In [ ]:
# Convert Tokenized Reviews to Word2Vec Embeddings
df['w2v_features'] = df['tokenized'].apply(lambda tokens: get_average_word2vec(tokens, word2vec_model))

# Prepare Features and Labels
X = np.vstack(df['w2v_features'].values)  # Convert list of arrays to 2D array
y = df['sentiment']

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and Train Gaussian Naive Bayes
gnb_model = GaussianNB()
gnb_model.fit(X_train, y_train)

# Predict and Evaluate
y_pred = gnb_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.67      0.72      0.70      4961
    positive       0.70      0.66      0.68      5039

    accuracy                           0.69     10000
   macro avg       0.69      0.69      0.69     10000
weighted avg       0.69      0.69      0.69     10000



In [ ]:
import joblib  # Import joblib for saving the model
# Save the GNB model
joblib.dump(gnb_model, 'GNB_model_word2vec.pkl')  # Save the trained RandomForest model
joblib.dump(word2vec_model, 'word2vec_model.pkl')  # Save the Word2Vec model (if needed)

['word2vec_model.pkl']

In [ ]:
import numpy as np
from gensim.models import Word2Vec
import joblib
import re

# Function to preprocess and tokenize the review
def tokenize(text):
    # Remove non-alphabet characters and split the text into words
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove anything that isn't a letter or space
    text = text.lower()  # Convert to lowercase
    return text.split()  # Split into words

# Function to process and convert the review to Word2Vec embedding
def get_average_word2vec(review, model, vector_size=100):
    words = review
    word_vecs = [model.wv[word] for word in words if word in model.wv]
    if len(word_vecs) == 0:
        return np.zeros(vector_size)
    return np.mean(word_vecs, axis=0)

# Load the pre-trained models
word2vec_model = Word2Vec.load('word2vec_model')  # Load Word2Vec model using gensim
gnb_model = joblib.load('GNB_model_word2vec.pkl')  # Load Naive Bayes model using joblib

# Example new review
new_review = 'I really enjoyed this movie. It was so good!'

# Tokenize the new review (using our custom tokenizer)
new_review_tokenized = tokenize(new_review)

# Convert the new review to its average Word2Vec embedding
new_review_vector = get_average_word2vec(new_review_tokenized, word2vec_model)

# Predict the sentiment using the trained Gaussian Naive Bayes (GNB) model
new_prediction = gnb_model.predict([new_review_vector])

# Print the predicted sentiment
print(f"Predicted Sentiment: {'Positive' if new_prediction[0] == 1 else 'Negative'}")


Predicted Sentiment: Negative


In [ ]:
# Example new review
new_review = 'Movie is awesome!'

# Tokenize the new review (using our custom tokenizer)
new_review_tokenized = tokenize(new_review)

# Convert the new review to its average Word2Vec embedding
new_review_vector = get_average_word2vec(new_review_tokenized, word2vec_model)

# Predict the sentiment using the trained Gaussian Naive Bayes (GNB) model
new_prediction = gnb_model.predict([new_review_vector])

# Print the predicted sentiment
print(f"Predicted Sentiment: {'Positive' if new_prediction[0] == 1 else 'Negative'}")


Predicted Sentiment: Negative


In [ ]:
# Example new review
new_review = 'Movie is worst!'

# Tokenize the new review (using our custom tokenizer)
new_review_tokenized = tokenize(new_review)

# Convert the new review to its average Word2Vec embedding
new_review_vector = get_average_word2vec(new_review_tokenized, word2vec_model)

# Predict the sentiment using the trained Gaussian Naive Bayes (GNB) model
new_prediction = gnb_model.predict([new_review_vector])

# Print the predicted sentiment
print(f"Predicted Sentiment: {'Positive' if new_prediction[0] == 1 else 'Negative'}")

Predicted Sentiment: Negative
